In [ ]:
!git clone https://github.com/thanhnew2001/songcover
%cd /content/songcover

In [ ]:
!pip install -r requirements.txt

In [ ]:
# Define all the functions:
# -*- coding: utf-8 -*-
import os
import subprocess
import json
import numpy as np
from scipy.io import wavfile
from tqdm import tqdm
from datetime import datetime, timedelta
from pydub import AudioSegment

# Separate audio using Demucs
def separate_audio(input_audio):
    command = f"demucs --two-stems=vocals {input_audio}"

    print("Separating audio into vocals and instruments...")
    result = subprocess.run(command.split(), stdout=subprocess.PIPE)
    print(result.stdout.decode())
    print("Audio separation completed.")

def GetTime(video_seconds):
    if video_seconds < 0:
        return "00:00:00.001"
    else:
        sec = timedelta(seconds=float(video_seconds))
        d = datetime(1, 1, 1) + sec
        return f"{str(d.hour).zfill(2)}:{str(d.minute).zfill(2)}:{str(d.second).zfill(2)}.001"

def windows(signal, window_size, step_size):
    for i_start in range(0, len(signal), step_size):
        i_end = i_start + window_size
        if i_end >= len(signal):
            break
        yield signal[i_start:i_end]

def energy(samples):
    return np.sum(np.power(samples, 2.)) / float(len(samples))

def rising_edges(binary_signal):
    previous_value = 0
    index = 0
    for x in binary_signal:
        if x and not previous_value:
            yield index
        previous_value = x
        index += 1

def split_audio(input_file, output_dir, min_silence_length=0.6, silence_threshold=1e-4, step_duration=0.003):
    os.makedirs(output_dir, exist_ok=True)

    print(f"Splitting audio file {input_file}...")
    sample_rate, samples = wavfile.read(input_file)
    max_amplitude = np.iinfo(samples.dtype).max
    max_energy = energy([max_amplitude])
    window_size = int(min_silence_length * sample_rate)
    step_size = int(step_duration * sample_rate)

    signal_windows = windows(samples, window_size, step_size)
    window_energy = (energy(w) / max_energy for w in tqdm(signal_windows))
    window_silence = (e > silence_threshold for e in window_energy)
    cut_times = (r * step_duration for r in rising_edges(window_silence))

    cut_samples = [int(t * sample_rate) for t in cut_times]
    cut_samples.append(-1)
    cut_ranges = [(i, cut_samples[i], cut_samples[i + 1]) for i in range(len(cut_samples) - 1)]

    video_sub = {str(i): [str(GetTime(cut_samples[i] / sample_rate)),
                          str(GetTime(cut_samples[i + 1] / sample_rate))]
                  for i in range(len(cut_samples) - 1)}

    for i, start, stop in tqdm(cut_ranges):
        output_file_path = os.path.join(output_dir, f"{os.path.splitext(os.path.basename(input_file))[0]}_{i:03d}.wav")
        wavfile.write(output_file_path, rate=sample_rate, data=samples[start:stop])
        print(f"Written file: {output_file_path}")

    with open(os.path.join(output_dir, f"{os.path.splitext(os.path.basename(input_file))[0]}.json"), 'w') as output:
        json.dump(video_sub, output)

    print("Audio splitting completed.")


# Preprocessing step
def preprocess():
    print("Preprocessing for training...")
    subprocess.run(['svc', 'pre-resample'])
    subprocess.run(['svc', 'pre-config'])

    subprocess.run(['svc', 'pre-hubert', '-fm', 'dio'])

    print("Preprocessing completed.")

# Training step
def train_model():
    print("Training model...")
    subprocess.run(['svc', 'train', '--model-path', 'logs/44k'])
    print("Training completed.")

# Combine vocal and instrument
def combine_audio(vocal_file, instrument_file, output_file):
    print("Combining vocal and instrument...")
    sound1 = AudioSegment.from_file(vocal_file)
    sound2 = AudioSegment.from_file(instrument_file)
    combined = sound1.overlay(sound2)
    combined.export(output_file, format='wav')
    print(f"Combined audio saved to {output_file}")

speaker_name = "thanh"
audio_input = "thanh.wav"  # Replace with your MP3 path
separate_audio(audio_input)

output_dir = f"dataset_raw/{speaker_name}"
split_audio(f"separated/htdemucs/{speaker_name}/vocals.wav", output_dir)

preprocess()  # Preprocess the data
train_model()  # Train the model


# Download a song you want to cover

In [ ]:
#@title Download Youtube WAV
from __future__ import unicode_literals
import yt_dlp
import ffmpeg
import sys

ydl_opts = {
    'format': 'bestaudio/best',
#    'outtmpl': 'output.%(ext)s',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'wav',
    }],
    "outtmpl": 'tan',  # this is where you can edit how you'd like the filenames to be formatted
}
def download_from_url(url):
    ydl.download([url])
    # stream = ffmpeg.input('output.m4a')
    # stream = ffmpeg.output(stream, 'output.wav')


with yt_dlp.YoutubeDL(ydl_opts) as ydl:
      url = "https://www.youtube.com/watch?v=GpmOn4RyzZI" #@param {type:"string"}
      download_from_url(url)

# Cover this song by your own voice

In [ ]:
# Import necessary libraries
import subprocess
from pydub import AudioSegment

VOICE = "tan"
# Define audio input
AUDIO_INPUT = "tan.wav"  # Path to the input audio file

# Separate vocals and instruments using Demucs
def separate_audio(input_audio):
    command = f"demucs --two-stems=vocals {input_audio}"
    result = subprocess.run(command.split(), stdout=subprocess.PIPE)
    print(result.stdout.decode())

# Run the separation function
separate_audio(AUDIO_INPUT)

# Inference
AUDIO = f"separated/htdemucs/{VOICE}/vocals"  # Path to the separated vocal file
MODEL = "logs/44k/G_199.pth"  # Path to the model file
CONFIG = "logs/44k/config.json"  # Path to the configuration file

# Change according to your voice tone
PITCH = 0  # Pitch adjustment (12 = 1 octave, -12 = -1 octave)

# Run inference
def run_inference(audio, config, model, pitch):
    print("Running inference...")
    command = f"svc infer {audio}.wav -c {config} -m {model} -na -t {pitch}"
    subprocess.run(command.split())
    print("Completed inference...")

run_inference(AUDIO, CONFIG, MODEL, PITCH)

# Combine vocal and instrument (song cover)
VOCAL = f"separated/htdemucs/{VOICE}/vocals.out.wav"  # Path to the vocal output
INSTRUMENT = f"separated/htdemucs/{VOICE}/no_vocals.wav"  # Path to the instrumental output

def combine_audio(vocal_file, instrument_file, output_file):
    sound1 = AudioSegment.from_file(vocal_file)
    sound2 = AudioSegment.from_file(instrument_file)
    sound1 = sound1 + (sound1.dBFS * 0.5)  # Increase by 50% of the current dBFS level

    combined = sound1.overlay(sound2)
    combined.export(output_file, format='wav')
    print(f"Combined audio saved to {output_file}")

# Run the combining function
combine_audio(VOCAL, INSTRUMENT, f"{VOICE}_covered.wav")

# Optional: play the final cover audio (if using an environment that supports audio pl